## MONGO AND NEAR BY QUERIES

In [1]:
from pymongo import MongoClient
import folium
import pandas as pd
import matplotlib
import numpy as np
import pandas as pd
import os
import requests
import json
from pandas.io.json import json_normalize
import re
from SRC.formulas_calculo import*

In [2]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

> The first thing Im gonna do, is import from Mongo all the data frames that ill be working with

In [3]:
cur =list(db.chosen_offices.find({},{"name":1,"latitude":1,"longitude":1, "geopoint":1}))
office_location = pd.DataFrame(cur)
office_location

,_id,name,latitude,longitude,geopoint
0,5efa38294c4b4802bbdf6e7c,Kongregate,37.786942,-122.401245,"{'type': 'Point', 'coordinates': [-122.401245,..."
1,5efa38294c4b4802bbdf6e7d,Curse,37.787092,-122.399972,"{'type': 'Point', 'coordinates': [-122.399972,..."
2,5efa38294c4b4802bbdf6e7e,Kabam,37.394456,-122.078531,"{'type': 'Point', 'coordinates': [-122.0785314..."
3,5efa38294c4b4802bbdf6e7f,Zynga,37.765158,-122.404234,"{'type': 'Point', 'coordinates': [-122.404234,..."
4,5efa38294c4b4802bbdf6e80,KIXEYE,NaN,NaN,None


In [4]:
cur =list(db.nightclubs.find({},{"name":1,"latitude":1,"longitude":1, "geopoint":1}))
nightclubs_location = pd.DataFrame(cur)
nightclubs_location.head()


,_id,name,latitude,longitude,geopoint
0,5efa386a51051c08f12ccb4f,[The Grand],37.779100,-122.397900,"{'type': 'Point', 'coordinates': [-122.3979003..."
1,5efa386a51051c08f12ccb50,[Smuggler's Cove],37.779403,-122.423372,"{'type': 'Point', 'coordinates': [-122.4233722..."
2,5efa386a51051c08f12ccb51,[Edinburgh Castle Pub],37.786127,-122.418957,"{'type': 'Point', 'coordinates': [-122.4189571..."
3,5efa386a51051c08f12ccb52,[Zero Zero],37.781636,-122.402083,"{'type': 'Point', 'coordinates': [-122.402083,..."
4,5efa386a51051c08f12ccb53,[The Chapel],37.760486,-122.421298,"{'type': 'Point', 'coordinates': [-122.4212979..."


In [5]:
cur =list(db.starbucks_place.find({},{"name":1,"latitude":1,"longitude":1, "geopoint":1}))
starbucks_location = pd.DataFrame(cur)
starbucks_location.head()

,_id,name,latitude,longitude,geopoint
0,5efa38849db4a6342ef5b134,[Starbucks],37.796647,-122.394850,"{'type': 'Point', 'coordinates': [-122.39485, ..."
1,5efa38849db4a6342ef5b135,[Starbucks],37.784124,-122.407653,"{'type': 'Point', 'coordinates': [-122.407653,..."
2,5efa38849db4a6342ef5b136,[Starbucks],37.789146,-122.408456,"{'type': 'Point', 'coordinates': [-122.408456,..."
3,5efa38849db4a6342ef5b137,[Starbucks],37.786261,-122.453194,"{'type': 'Point', 'coordinates': [-122.453194,..."
4,5efa38849db4a6342ef5b138,[Starbucks],37.784855,-122.399996,"{'type': 'Point', 'coordinates': [-122.3999959..."


In [6]:
cur =list(db.vegan.find({},{"name":1,"latitude":1,"longitude":1, "geopoint":1}))
vegan_location = pd.DataFrame(cur)
vegan_location.head()

,_id,name,latitude,longitude,geopoint
0,5efa38afaf923e76c3898dce,[The Flying Falafel],37.781266,-122.411230,"{'type': 'Point', 'coordinates': [-122.4112304..."
1,5efa38afaf923e76c3898dcf,[Golden Era],37.781318,-122.416697,"{'type': 'Point', 'coordinates': [-122.4166966..."
2,5efa38afaf923e76c3898dd0,[Indochine Vegan],37.764626,-122.422160,"{'type': 'Point', 'coordinates': [-122.4221595..."
3,5efa38afaf923e76c3898dd1,[Ananda Fuara],37.777875,-122.416294,"{'type': 'Point', 'coordinates': [-122.416294,..."
4,5efa38afaf923e76c3898dd2,[Loving Hut],37.784307,-122.406012,"{'type': 'Point', 'coordinates': [-122.406012,..."


In [7]:
cur =list(db.veterinary_place.find({},{"name":1,"latitude":1,"longitude":1, "geopoint":1}))
veterinary_location = pd.DataFrame(cur)
veterinary_location.head()

,_id,name,latitude,longitude,geopoint
0,5efa389ffe122b1cba544978,[VCA San Francisco Veterinary Specialists],37.761480,-122.412279,"{'type': 'Point', 'coordinates': [-122.4122788..."
1,5efa389ffe122b1cba544979,[San Francisco SPCA Veterinary Hospital Pacifi...,37.791284,-122.434625,"{'type': 'Point', 'coordinates': [-122.434625,..."
2,5efa389ffe122b1cba54497a,[Ocean Avenue Veterinary Hospital],37.723144,-122.453253,"{'type': 'Point', 'coordinates': [-122.4532528..."
3,5efa389ffe122b1cba54497b,[Park Animal Hospital],37.765639,-122.466607,"{'type': 'Point', 'coordinates': [-122.466607,..."
4,5efa389ffe122b1cba54497c,[San Francisco Pet Hospital],37.776581,-122.439369,"{'type': 'Point', 'coordinates': [-122.4393694..."


## QUERY TO GEOCODE

> Next, using the functions geoQueryNear and a For loop I will try to find out the nearby starbucks from my desire offices. In other words, figure out how many starbucks are in a 500m radious from each office location

In [24]:
starbucks = []
for i in range(len(starbucks_location)):
    q = db.chosen_offices.find(geoQueryNear(starbucks_location.iloc[i].geopoint, radius=500),{"_id":0,"company_id":0})
    near_offices = list(q)
    starbucks.append(near_offices)
starbucks


[[],
 [],
 [],
 [],
 [{'name': 'Curse',
   'category_code': 'games_video',
   'total_money_raised': '$12M',
   'address1': '60 Broadway',
   'city': 'San Francisco',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.787092,
   'longitude': -122.399972,
   'geopoint': {'type': 'Point', 'coordinates': [-122.399972, 37.787092]}},
  {'name': 'Kongregate',
   'category_code': 'games_video',
   'total_money_raised': '$19M',
   'address1': '2 Shaw Alley',
   'city': 'San Francisco',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.786942,
   'longitude': -122.401245,
   'geopoint': {'type': 'Point', 'coordinates': [-122.401245, 37.786942]}}],
 [],
 [],
 [],
 [],
 [{'name': 'Kongregate',
   'category_code': 'games_video',
   'total_money_raised': '$19M',
   'address1': '2 Shaw Alley',
   'city': 'San Francisco',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.786942,
   'longitude': -122.401245,
   'geopoint': {'type': 'Point', 'coordinates

> The list needs to be flatten in order to convert it into a dataframe

In [11]:
starbucks_close = [data for enterprise in starbucks for data in enterprise]
starbucks_close = pd.DataFrame(starbucks_close)
starbucks_close["name"].value_counts()

Curse         2
Kongregate    2
Name: name, dtype: int64

> I will repeat the same procedure for: 
    - NightClubs
    - Vegan Restaurants
    - Veterinaries

In [12]:
nightclubs = []
for i in range(len(nightclubs_location)):
    q = db.chosen_offices.find(geoQueryNear(nightclubs_location.iloc[i].geopoint, radius=3000),{"_id":0,"company_id":0})
    near_offices = list(q)
    nightclubs.append(near_offices)
nightclubs[0]

[{'name': 'Curse',
  'category_code': 'games_video',
  'total_money_raised': '$12M',
  'address1': '60 Broadway',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.787092,
  'longitude': -122.399972,
  'geopoint': {'type': 'Point', 'coordinates': [-122.399972, 37.787092]}},
 {'name': 'Kongregate',
  'category_code': 'games_video',
  'total_money_raised': '$19M',
  'address1': '2 Shaw Alley',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.786942,
  'longitude': -122.401245,
  'geopoint': {'type': 'Point', 'coordinates': [-122.401245, 37.786942]}},
 {'name': 'Zynga',
  'category_code': 'games_video',
  'total_money_raised': '$860M',
  'address1': '365 Vermont St.',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.765158,
  'longitude': -122.404234,
  'geopoint': {'type': 'Point', 'coordinates': [-122.404234, 37.765158]}}]

In [13]:
nightclubs_close = [data for enterprise in nightclubs for data in enterprise]
nightclubs_close = pd.DataFrame(nightclubs_close)
nightclubs_close["name"].value_counts()

Zynga         18
Curse         16
Kongregate    16
Name: name, dtype: int64

In [14]:
vegan = []
for i in range(len(vegan_location)):
    q = db.chosen_offices.find(geoQueryNear(vegan_location.iloc[i].geopoint, radius=1500),{"_id":0,"company_id":0})
    near_offices = list(q)
    vegan.append(near_offices)
vegan[0]

[{'name': 'Kongregate',
  'category_code': 'games_video',
  'total_money_raised': '$19M',
  'address1': '2 Shaw Alley',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.786942,
  'longitude': -122.401245,
  'geopoint': {'type': 'Point', 'coordinates': [-122.401245, 37.786942]}},
 {'name': 'Curse',
  'category_code': 'games_video',
  'total_money_raised': '$12M',
  'address1': '60 Broadway',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.787092,
  'longitude': -122.399972,
  'geopoint': {'type': 'Point', 'coordinates': [-122.399972, 37.787092]}}]

In [15]:
vegan_close = [data for enterprise in vegan for data in enterprise]
vegan_close = pd.DataFrame(vegan_close)
vegan_close.head()
vegan_close["name"].value_counts()

Kongregate    5
Curse         3
Zynga         2
Name: name, dtype: int64

In [16]:
veterinary = []
for i in range(len(veterinary_location)):
    q = db.chosen_offices.find(geoQueryNear(veterinary_location.iloc[i].geopoint, radius=5000),{"_id":0,"company_id":0})
    near_offices = list(q)
    veterinary.append(near_offices)
veterinary[0]

[{'name': 'Zynga',
  'category_code': 'games_video',
  'total_money_raised': '$860M',
  'address1': '365 Vermont St.',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.765158,
  'longitude': -122.404234,
  'geopoint': {'type': 'Point', 'coordinates': [-122.404234, 37.765158]}},
 {'name': 'Kongregate',
  'category_code': 'games_video',
  'total_money_raised': '$19M',
  'address1': '2 Shaw Alley',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.786942,
  'longitude': -122.401245,
  'geopoint': {'type': 'Point', 'coordinates': [-122.401245, 37.786942]}},
 {'name': 'Curse',
  'category_code': 'games_video',
  'total_money_raised': '$12M',
  'address1': '60 Broadway',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.787092,
  'longitude': -122.399972,
  'geopoint': {'type': 'Point', 'coordinates': [-122.399972, 37.787092]}}]

In [17]:
veterinary_close = [data for enterprise in veterinary for data in enterprise]
veterinary_close = pd.DataFrame(veterinary_close)
veterinary_close["name"].value_counts()

Zynga         12
Kongregate    11
Curse         11
Name: name, dtype: int64

## FOLIUM
    


In [18]:
!pip3 install folium

In [19]:
from folium import Map, Marker, Icon, FeatureGroup, LayerControl, Choropleth
from folium.plugins import HeatMap
import pandas as pd

In [20]:
# Offices that have a veterinary at 5000m
m = Map([37.773972,-122.431297],zoom_start=12)    
for i, row in veterinary_close.iterrows():
    icon = Icon(
                color="orange",
                prefix="fa",
                icon="paw"
            )
    Marker(
        location=row[["latitude","longitude"]],
        icon=icon
    ).add_to(m)
m

In [21]:
# Offices that have a night club at 3000 m
m = Map([37.773972,-122.431297],zoom_start=12)

for i, row in nightclubs_close.iterrows():
    icon = Icon(
                color="blue",
                prefix="fa",
                icon="beer"
            )
    Marker(
        location=row[["latitude","longitude"]],
        icon=icon
    ).add_to(m)
m

In [22]:
# Offices that have a vegan restaurant at 1500m
m = Map([37.773972,-122.431297],zoom_start=12)

for i, row in vegan_close.iterrows():
    icon = Icon(
                color="red",
                prefix="fa",
                icon="hamburger"
            )
    Marker(
        location=row[["latitude","longitude"]],
        icon=icon
    ).add_to(m)
m

In [23]:
# Offices that have a starbucks at 500m
m = Map([37.773972,-122.431297],zoom_start=12)

for i, row in starbucks_close.iterrows():
    icon = Icon(
                color="green",
                prefix="fa",
                icon="coffe"
            )
    Marker(
        location=row[["latitude","longitude"]],
        icon=icon
    ).add_to(m)
m

## Chosen Offices 

 > Only 2 locations have all the characteristics and establishments that our enterprise needs
 